## Part 1: Explore the Data

Import the data and use Pandas to learn more about the dataset.

In [ ]:
import pandas as pd

df = pd.read_csv('Resources/client_dataset.csv')

df.head()

In [ ]:
# View the column names in the data

df.columns

In [ ]:
# Use the describe function to gather some basic statistics

df.describe()

In [ ]:
# Use this space to do any additional research
# and familiarize yourself with the data.

## common commands to get additional information about the data

df.shape

In [ ]:
df.info()

In [ ]:
df.count()

In [ ]:
## notes to self:

## most methods return a vew of the original data, to make a copy:
## deepcopy = df.copy()
## shallowcopy = df.copy(deep=False)

## ("row", "column") notation in loc and iloc
## iloc needs index values
## loc row data and column name

In [ ]:
# What three item categories had the most entries?

cat_counts = df["category"].value_counts()
## To try to make my output match perfectly, adjusting a few series attributes
cat_counts.index.name = None
cat_counts.name = "category"
cat_counts.head(3)

In [ ]:
# For the category with the most entries,
# which subcategory had the most entries?

## filter dataframe for just top category (cat_counts.index[0])
top_cat = df.loc[ df["category"] == cat_counts.index[0]]
## and count same as before
top_cat_counts = top_cat["subcategory"].value_counts()
## To try to make my output match perfectly, adjusting a few series attributes
top_cat_counts.index.name = None
top_cat_counts.name = "subcategory"
top_cat_counts.head(1)

In [ ]:
# Which five clients had the most entries in the data?

top_clients = df["client_id"].value_counts().head(5)
## To try to make my output match perfectly, adjusting a few series attributes
top_clients.index.name = None
top_clients.name = "client_id"
top_clients

In [ ]:
# Store the client ids of those top 5 clients in a list.

top_cli_list = list(top_clients.index)
top_cli_list

In [ ]:
# How many total units (the qty column) did the
# client with the most entries order order?

## filter dataframe for just top client (top_cli_list[0])
top_client = df.loc[ df["client_id"] == top_cli_list[0]]
## sum the qty column of filtered
top_qty = top_client["qty"].sum()
top_qty

## Part 2: Transform the Data
Do we know that this client spent the more money than client 66037? If not, how would we find out? Transform the data using the steps below to prepare it for analysis.

In [ ]:
# Create a column that calculates the 
# subtotal for each line using the unit_price
# and the qty

## append subtotal column to full data set
df["line_subtotal"] = df["unit_price"] * df["qty"]

## display only snippet based on starter code file
df[["unit_price" , "qty", "line_subtotal"]].head(2)

In [ ]:
# Create a column for shipping price.
# Assume a shipping price of $7 per pound
# for orders over 50 pounds and $10 per
# pound for items 50 pounds or under.

## first calcualte total weights
df["total_weight"] = df["unit_weight"] * df["qty"]

## could perhaps use a lambda function here to distinguish weight differnces
## but the logic on this two step solution was easier for me

## append shipping column and set to $10/lb for anything total weight 50lb and under
df.loc[df["total_weight"] <= 50, "shipping_price"] = df["total_weight"] * 10

## now set anything total weight over 50lb to shipping $7/lb
df.loc[df["total_weight"] > 50, "shipping_price"] = df["total_weight"] * 7

## display snippet based on starter file output
df[["unit_price", "unit_weight", "qty", "total_weight", "shipping_price"]].head(3)

In [ ]:
# Create a column for the total price
# using the subtotal and the shipping price
# along with a sales tax of 9.25%

## append total (line price) column with rounded sum of subtotal, shipping, tax
df["line_price"] = round((df["line_subtotal"] + df["shipping_price"]) * 1.0925, 2)

## and display snippet based on starter file output
df[["line_subtotal", "shipping_price", "line_price"]].head(3)

In [ ]:
# Create a column for the cost
# of each line using unit cost, qty, and
# shipping price (assume the shipping cost
# is exactly what is charged to the client).

## append manufacturing total (line cost) column with appropriate values
df["line_cost"] = (df["unit_cost"] * df["qty"]) + df["shipping_price"]

## again display as per original
df.head(3)

In [ ]:
# Create a column for the profit of
# each line using line cost and line price

## append calculated profit column
df["line_profit"] = df["line_price"] - df["line_cost"]

## again display as per original
df.head(3)

## Part 3: Confirm your work
You have email receipts showing that the total prices for 3 orders. Confirm that your calculations match the receipts. Remember, each order has multiple lines.

Order ID 2742071 had a total price of \$152,811.89

Order ID 2173913 had a total price of \$162,388.71

Order ID 6128929 had a total price of \$923,441.25


In [ ]:
# Check your work using the totals above

## sum all approrpriate rows for each of the IDs
id2742071 = round(sum(df.loc[df["order_id"] == 2742071]["line_price"]), 2)
id2173913 = round(sum(df.loc[df["order_id"] == 2173913]["line_price"]), 2)
id6128929 = round(sum(df.loc[df["order_id"] == 6128929]["line_price"]), 2)

## finally format the value check
print(f"""Order 2742071 total: ${id2742071}
Order 2173913 total: ${id2173913}
Order 6128929 total: ${id6128929}""")

## Part 4: Summarize and Analyze
Use the new columns with confirmed values to find the following information.

In [ ]:
# How much did each of the top 5 clients by quantity
# spend? Check your work from Part 1 for client ids.

## Loop through top clients from Part 1 and sum line totals then print
for client in top_cli_list:
    cli_total = round(sum(df.loc[df["client_id"] == client]["line_price"]), 2)
    print(f"{client}: ${cli_total}")

In [ ]:
# Create a summary DataFrame showing the totals for the
# for the top 5 clients with the following information:
# total units purchased, total shipping price,
# total revenue, and total profit. Sort by total profit.

## loop through top clients and total rows setting up list of dictionaries for new dataframe
dict_lst = []
for client in top_cli_list:
    total_units = sum(df.loc[df["client_id"] == client]["qty"])
    total_shipping = round(sum(df.loc[df["client_id"] == client]["shipping_price"]), 2)
    total_revenue = round(sum(df.loc[df["client_id"] == client]["line_price"]), 2)
    total_profit = round(sum(df.loc[df["client_id"] == client]["line_profit"]), 2)
    ## oops didn't see this in the directions, but it is on the original output
    total_cost = round(sum(df.loc[df["client_id"] == client]["line_cost"]), 2)
    ## could have put the nicer column names here and skip some the next step
    ## but I want my output to perfectly match the original
    client_totals = {"client_id" : client,
                     "qty" : total_units,
                     "shipping_price" : total_shipping,
                     "line_price" : total_revenue,
                     "line_cost" : total_cost,
                     "line_profit" : total_profit
    }
    dict_lst.append(client_totals)

## make summary dataframe from list of dictionaries
summary_df = pd.DataFrame(dict_lst)

## sort it by total profit
summary_df.sort_values("line_profit", ascending=False, inplace=True)

## and output full smmary table
summary_df

In [ ]:
# Format the data and rename the columns
# to names suitable for presentation.
# Currency should be in millions of dollars.

## when there are few columns this method is easier than .rename()
summary_df.columns = ["Client ID", "Units", "Shipping", "Total Revenue", "Total Cost", "Total Profit"]

## simplify each column values to millions
summary_df["Shipping"] = round(summary_df["Shipping"]/1000000, 2)
summary_df["Total Revenue"] = round(summary_df["Total Revenue"]/1000000, 2)
summary_df["Total Cost"] = round(summary_df["Total Cost"]/1000000, 2)
summary_df["Total Profit"] = round(summary_df["Total Profit"]/1000000, 2)

## Convert colum types to string
summary_df["Shipping"] = summary_df["Shipping"].astype("string")
summary_df["Total Revenue"] = summary_df["Total Revenue"].astype("string")
summary_df["Total Cost"] = summary_df["Total Cost"].astype("string")
summary_df["Total Profit"] = summary_df["Total Profit"].astype("string")

## function to add '$' and 'M' to all, and maybe '0'
def add_txt(amount):
    if amount[-2] == '.':
        # apparently astype("string") drops ending 0
        amount_str = "$" + amount + "0M"
    else:
        amount_str = "$" + amount + "M"
    return amount_str

## apply the add_text function to each column
summary_df["Shipping"] = summary_df["Shipping"].apply(add_txt)
summary_df["Total Revenue"] = summary_df["Total Revenue"].apply(add_txt)
summary_df["Total Cost"] = summary_df["Total Cost"].apply(add_txt)
summary_df["Total Profit"] = summary_df["Total Profit"].apply(add_txt)

In [ ]:
# Sort the updated data by "Total Profit" form highest to lowest

## Not sure this is really needed as I sorted "inplace" above
summary_df.sort_values("Total Profit", ascending=False, inplace=True)

## and final output full summar table
summary_df

# Summary
While client 33615 ordered items most frequently, we find that the next four clients by number of orders spent a lot more money.  In fact, client 24741 spent more than six times the next closest client, 38378, and nearly 10 times that of client 33615.  We also made an 44% profit margin on sales to client 24741, better than the 25–32% margin of the other top clients.  Finally, I would suggest we find a new shipper or negotiate a better rate.  When our clients are spending multi-millions in shipping costs alone, we are doing them a disservice.